In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv


In [2]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")



In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.over_sampling import SMOTE
from collections import Counter

In [4]:
df_test = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv")
df_test_l = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv").replace(-1,0)
print(df_test.shape)
df_test = pd.merge(df_test, df_test_l, how="left", on = "id")
print(df_test.shape)

(153164, 2)
(153164, 8)


In [5]:
df = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv")
print(df.shape)
df = pd.concat([df, df_test])
print(df.shape)
del df_test

(159571, 8)
(312735, 8)


In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import pandas as pd

def preprocess_text(comment_text):
    
    
    # Remove links
    comment_text = re.sub(r'http\S+', '', comment_text)
    comment_text = comment_text.replace('\n', ' ')
    
    # Additional text cleaning
    comment_text = comment_text.lower()
    comment_text = re.sub(r"what's", "what is ", comment_text)
    comment_text = re.sub(r"\'s", " ", comment_text)
    comment_text = re.sub(r"\'ve", " have ", comment_text)
    comment_text = re.sub(r"can't", "cannot ", comment_text)
    comment_text = re.sub(r"n't", " not ", comment_text)
    comment_text = re.sub(r"i'm", "i am ", comment_text)
    comment_text = re.sub(r"\'re", " are ", comment_text)
    comment_text = re.sub(r"\'d", " would ", comment_text)
    comment_text = re.sub(r"\'ll", " will ", comment_text)
    comment_text = re.sub(r"\'scuse", " excuse ", comment_text)
    comment_text = re.sub('\W', ' ', comment_text)
    comment_text = re.sub('\s+', ' ', comment_text)
    comment_text = comment_text.strip(' ')

    # Tokenize the text
    words = word_tokenize(comment_text)
    
    # Remove digits
    words = [word for word in words if not word.isdigit()]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Join the words back into a string
    cleaned_text = ' '.join(words)
    
    return cleaned_text


In [7]:
df['cleaned_text'] = df['comment_text'].apply(preprocess_text)

# Split the data into input features and target variable
df['y'] = (df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) > 0).astype(int)
df = df[['comment_text', 'y']].rename(columns={'comment_text': 'text'})

tfidf_vectorizer = TfidfVectorizer(max_features=500000)
X = tfidf_vectorizer.fit_transform(df['text'])


y = df['y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

os = SMOTE()
X_Otrain, y_Otrain = os.fit_resample(X_train, y_train)
count = Counter(y_Otrain)
print(count)

Counter({0: 203228, 1: 203228})


In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
clf = GridSearchCV(MultinomialNB(),{
    'alpha': [0,0.0001,0.001,0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
    'fit_prior': [True,False]
}, cv=10, return_train_score=False,n_jobs=-1)
clf.fit(X_Otrain,y_Otrain)
df = pd.DataFrame(clf.cv_results_)
df

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_fit_prior,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.426781,0.044434,0.032419,0.001787,0,True,"{'alpha': 0, 'fit_prior': True}",0.910225,0.907642,0.908650,0.910053,0.907420,0.908380,0.908377,0.907738,0.908697,0.910346,0.908753,0.001036,1
1,0.389593,0.006223,0.033211,0.001430,0,False,"{'alpha': 0, 'fit_prior': False}",0.910225,0.907642,0.908650,0.910053,0.907420,0.908380,0.908328,0.907442,0.908697,0.910346,0.908718,0.001070,2
2,0.392556,0.004915,0.032580,0.001570,0.0001,True,"{'alpha': 0.0001, 'fit_prior': True}",0.908035,0.905673,0.906559,0.907666,0.905575,0.906215,0.906188,0.905400,0.906852,0.907959,0.906612,0.000937,3
3,0.394591,0.005657,0.032268,0.002172,0.0001,False,"{'alpha': 0.0001, 'fit_prior': False}",0.908035,0.905673,0.906559,0.907666,0.905575,0.906215,0.906139,0.905105,0.906852,0.907959,0.906578,0.000980,4
4,0.397366,0.004705,0.033151,0.000989,0.001,True,"{'alpha': 0.001, 'fit_prior': True}",0.905821,0.903361,0.904074,0.905649,0.903115,0.904271,0.903949,0.903309,0.904835,0.906163,0.904455,0.001053,5
5,0.395362,0.002268,0.033670,0.001062,0.001,False,"{'alpha': 0.001, 'fit_prior': False}",0.905821,0.903361,0.904074,0.905649,0.903115,0.904271,0.903900,0.903014,0.904835,0.906163,0.904420,0.001091,6
6,0.394059,0.003548,0.033719,0.001204,0.01,True,"{'alpha': 0.01, 'fit_prior': True}",0.899867,0.897382,0.898662,0.900556,0.896644,0.898785,0.897896,0.897970,0.899274,0.900406,0.898744,0.001232,7
7,0.436542,0.032243,0.041611,0.011656,0.01,False,"{'alpha': 0.01, 'fit_prior': False}",0.899867,0.897382,0.898662,0.900556,0.896644,0.898785,0.897847,0.897675,0.899274,0.900406,0.898710,0.001257,8
8,0.404508,0.019104,0.033738,0.001701,0.1,True,"{'alpha': 0.1, 'fit_prior': True}",0.885204,0.885032,0.884761,0.886680,0.884023,0.885106,0.884586,0.885349,0.886161,0.887169,0.885407,0.000926,9
9,0.388030,0.012846,0.033576,0.002236,0.1,False,"{'alpha': 0.1, 'fit_prior': False}",0.885204,0.885032,0.884761,0.886680,0.884023,0.885106,0.884537,0.885054,0.886185,0.887169,0.885375,0.000939,10


In [10]:
clf.best_score_

0.9087527312849382

In [11]:
training_score = clf.score(X_Otrain, y_Otrain)

print("Training Score:", training_score)

Training Score: 0.9303983703033046


In [12]:
from sklearn.metrics import classification_report
best_nb_model=clf.best_estimator_
y_pred = best_nb_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.87      0.91     87039
           1       0.25      0.58      0.35      6782

    accuracy                           0.85     93821
   macro avg       0.61      0.72      0.63     93821
weighted avg       0.91      0.85      0.87     93821



In [13]:
df[['param_alpha','param_fit_prior','mean_test_score']]

,param_alpha,param_fit_prior,mean_test_score
0,0,True,0.908753
1,0,False,0.908718
2,0.0001,True,0.906612
3,0.0001,False,0.906578
4,0.001,True,0.904455
5,0.001,False,0.904420
6,0.01,True,0.898744
7,0.01,False,0.898710
8,0.1,True,0.885407
9,0.1,False,0.885375


In [14]:
# Print the best parameters
print("Best parameters:", clf.best_params_)

Best parameters: {'alpha': 0, 'fit_prior': True}
